I am running 5 fold cross validation twice. Once using model_selection.cross_validate and the second time using a loop. I would expect the result to be the same or at least the difference to be very low. I am receiveing however mean log_loss 2.5263 in case of cross_validate function and log_loss 3.7644 in the case of the loop.  
Do you know why the difference appears?
As I checked in the documentation scoring='neg_log_loss' is equivalent of metrics.log_loss 

In [ ]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import cross_validate
import xgboost as xgb
from sklearn.model_selection import KFold
from sklearn.metrics import log_loss

In [ ]:
path='/kaggle/input/lish-moa/'
rnd_stat=10
CV_no=5

 # Data upload & preprocessing

In [ ]:
train_features=pd.read_csv(path+'train_features.csv',index_col='sig_id').sort_index()
train_targets=pd.read_csv(path+'train_targets_scored.csv',index_col='sig_id').sort_index()
test_features=pd.read_csv(path+'test_features.csv',index_col='sig_id').sort_index()

In [ ]:
le_cp_type=preprocessing.LabelEncoder()
le_cp_type.fit(train_features['cp_type'].values)
train_features['cp_type']=le_cp_type.transform(train_features['cp_type'])
test_features['cp_type']=le_cp_type.transform(test_features['cp_type'])

le_cp_dose=preprocessing.LabelEncoder()
le_cp_dose.fit(train_features['cp_dose'].values)
train_features['cp_dose']=le_cp_dose.transform(train_features['cp_dose'])
test_features['cp_dose']=le_cp_dose.transform(test_features['cp_dose'])

## Model definition

In [ ]:
model=xgb.XGBClassifier(n_estimators=200,
    n_jobs=-1,
    min_child_weight=7,
    max_depth=3,
    gamma=0.4,
    subsample=1,
    colsample_bytree=1,
    tree_method='gpu_hist',
    #tree_method='hist',
    learning_rate=0.05,
    reg_alpha=1,
    random_state=rnd_stat)
 
classif = OneVsRestClassifier(model,n_jobs=-1)

## Data split

In [ ]:
skf = KFold(n_splits=CV_no)
skf.get_n_splits(train_features, train_targets)

# Model_selection.cross_validate

In [ ]:
scores=cross_validate(classif,train_features, train_targets,scoring='neg_log_loss',cv=skf)
print('Mean',np.mean(scores['test_score']),'sd',np.std(scores['test_score']))
print(scores['test_score'])

# Own cross validation

In [ ]:
%%time
scores2=[]
n=0
for train_index, test_index in skf.split(train_features, train_targets):    
    full_prediction=pd.DataFrame(index=test_index,columns=train_targets.columns)
    n=n+1
    print('Run ',n,' of ',CV_no)
    classif.fit(train_features.iloc[train_index], train_targets.iloc[train_index])
    full_prediction=classif.predict_proba(train_targets.iloc[test_index]) 
    scores2.append(log_loss(train_targets.iloc[test_index],full_prediction))
print('Mean',np.mean(scores2),'sd',np.std(scores2))
print(scores2)